In [ ]:
# !pip install unsloth transformers datasets peft accelerate vllm

In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

import os
os.environ['TRITON_JIT_DISABLE_OPT'] = '1'

In [ ]:
MODEL_NAME = <specify your model name here>
TARGET_LAYER = <"full" or "QKVO">
LORA_ALPHA = 16
MAX_SEQUENCE_LENGTH = 2048


if TARGET_LAYER == "full":
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
elif TARGET_LAYER == "QKVO":
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]

mkdir: cannot create directory ‘data’: File exists


In [ ]:

# Load your dataset (replace with real path if needed)
df = pd.read_csv("/content/aggregated_clean_data.csv")

# Optional: quick clean
df = df.dropna(subset=["post_vi", "condition", "severity_score"])
df = df[df["post_vi"].str.strip() != ""]

# Create formatted instruction-style examples
def format_qwen_example(row):
    return {
        "input": row["post_vi"],
        "output": f'{{"condition": "{row["condition"]}", "severity_score": {int(row["severity_score"])}}}'
    }

formatted = df.apply(format_qwen_example, axis=1).tolist()

# Train/Val/Test split
train_data, temp_data = train_test_split(formatted, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)


train_path = "data/train_unsloth.jsonl"
val_path = "data/val_unsloth.jsonl"
test_path = "data/test_unsloth.jsonl"

def save_jsonl(data, path):
    with open(path, "w", encoding="utf-8") as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")

save_jsonl(train_data, train_path)
save_jsonl(val_data, val_path)
save_jsonl(test_data, test_path)


In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    device_map="auto",
    max_seq_length = MAX_SEQUENCE_LENGTH,
    dtype = torch.float16,
    load_in_4bit = True
)

tokenizer.pad_token = tokenizer.eos_token

#  Attach PEFT/LoRA adapters — THIS LINE FIXES THE ERROR
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_ALPHA,
    target_modules=target_modules
    use_gradient_checkpointing = False
)


==((====))==  Unsloth 2025.6.3: Fast Qwen3 patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.75G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.6.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:


# Load JSONL dataset
dataset = load_dataset("json", data_files={
    "train": "data/train_unsloth.jsonl",
    "validation": "data/val_unsloth.jsonl",
    "test": "data/test_unsloth.jsonl"
})

# Add EOS token to the end of output
def format_with_eos(example):
    return {
        "text": f"<|user|>\n{example['input']}\n<|assistant|>\n{example['output']}</s>"
    }

# Apply formatting
tokenized_dataset  = dataset.map(format_with_eos)

# Preview to confirm
tokenized_dataset["validation"][0]


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1466 [00:00<?, ? examples/s]

Map:   0%|          | 0/183 [00:00<?, ? examples/s]

Map:   0%|          | 0/184 [00:00<?, ? examples/s]

{'input': 'Cảnh báo:  Có người đang spam tin nhắn riêng quảng cáo blog mentorself.com nhắm vào cộng đồng mình. ###Cập nhật (Thứ Sáu, 9 tháng 3, 11:45 PST):  Người này giờ đã lập thêm vài tài khoản mới và dùng chat nhiều hơn tin nhắn riêng. Admin đã yêu cầu mọi người report những tin nhắn chat từ kẻ spam này cố gắng dụ mọi người vào mentorself.com. ### * Trên máy tính, các bạn có thể report bằng cách giữ chuột lên biểu tượng cờ. ### * Trên điện thoại (ít nhất là app chính thức trên Android), giữ lâu vào tin nhắn thì sẽ hiện lựa chọn "report". Cập nhật trước:  Mình đã được admin Reddit thông báo vấn đề này đã được xử lý, nhưng mình sẽ để bài này lên một thời gian nữa phòng trường hợp chúng nó lén lút quay lại. Nếu các bạn đăng bài hoặc bình luận rồi nhận được yêu cầu chat hoặc tin nhắn riêng tương tự như mô tả ở đây, [hãy báo cho mình biết](http://www.reddit.com/message/compose?to=%2Fr%2Fdepression). Nếu nhận được tin nhắn riêng hoặc chat sau khi đăng bài ở đây quảng cáo cho cái blog tự 

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=5,
    num_train_epochs=3,
    learning_rate=5e-4,
    fp16=True,
    logging_steps=10,
    output_dir="mental_health_model_qwen",
    save_strategy="epoch",
    save_total_limit=2,                        # ✅ Limit checkpoints
    lr_scheduler_type="cosine",                # ✅ Smoother learning rate
    remove_unused_columns=False,                # ✅ For safety with custom fields
    # disable_tqdm=True # uncomment this for faster training
    # Adding this line to disable Triton
    # use_cpu=True, # This forces the use of CPU, bypassing Triton
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    dataset_text_field="text",
    max_seq_length=MAX_SEQUENCE_LENGTH,
    args=training_args,
)

trainer.train()

Unsloth: Tokenizing ["text"]:   0%|          | 0/1466 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/183 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,466 | Num Epochs = 3 | Total steps = 552
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 43,646,976/8,000,000,000 (0.55% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tranhieu170600 (grace_ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.349200
20,1.958600
30,1.980700
40,2.008000
50,1.959900
60,1.937700
70,1.945800
80,1.930000
90,1.929900
100,1.911700


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=552, training_loss=1.621791408545729, metrics={'train_runtime': 3494.5373, 'train_samples_per_second': 1.259, 'train_steps_per_second': 0.158, 'total_flos': 4.836396708513792e+16, 'train_loss': 1.621791408545729})

In [ ]:
trainer.save_model("mental_detection_unsloth")
tokenizer.save_pretrained("mental_detection_unsloth")


('mental_detection_unsloth/tokenizer_config.json',
 'mental_detection_unsloth/special_tokens_map.json',
 'mental_detection_unsloth/vocab.json',
 'mental_detection_unsloth/merges.txt',
 'mental_detection_unsloth/added_tokens.json',
 'mental_detection_unsloth/tokenizer.json')

In [ ]:
text = "Tôi cảm thấy rất mệt mỏi, mất ngủ và không muốn nói chuyện với ai nữa."

input_text = f"<|user|>\n{text}\n<|assistant|>\n"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<|user|>
Tôi cảm thấy rất mệt mỏi, mất ngủ và không muốn nói chuyện với ai nữa.
<|assistant|>
{"condition": "depression", "severity_score": 3}</s> {"condition": "stress", "severity_score": 3}</s> {"condition": "stress", "severity_score": 3}</s> {"condition": "


In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'<|user|>\nTôi cảm thấy rất mệt mỏi, mất ngủ và không muốn nói chuyện với ai nữa.\n<|assistant|>\n{"condition": "depression", "severity_score": 3}</s> {"condition": "stress", "severity_score": 3}</s> {"condition": "stress", "severity_score": 3}</s> {"condition": "'

In [ ]:
import json
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

def extract_json_from_response(response_str):
    """
    Extract dictionary from model response.
    Assumes format is like: {"condition": "...", "severity_score": ...}
    """
    try:
        json_start = response_str.find("{")
        json_end = response_str.find("}") + 1
        json_str = response_str[json_start:json_end]
        return json.loads(json_str)
    except:
        return {"condition": "unknown", "severity_score": -1}


In [ ]:
# Step 1: Convert HuggingFace test dataset back to DataFrame
test_hf_df = tokenized_dataset["test"].to_pandas()

# Step 2: Match original 'post_vi' column by comparing with 'input'
# Merge on input text, which came from post_vi
merged_test_df = pd.merge(test_hf_df, df, left_on="input", right_on="post_vi")


In [ ]:
merged_test_df

,input,output,text,condition,post_vi,response_vi,severity_score
0,"Tôi là một người trưởng thành mắc bệnh tâm lý,...","{""condition"": ""stress"", ""severity_score"": 4}",<|user|>\nTôi là một người trưởng thành mắc bệ...,stress,"Tôi là một người trưởng thành mắc bệnh tâm lý,...",Ngôn ngữ được sử dụng trong bài viết cho thấy ...,4.0
1,Nhật ký: Hôm nay tôi đang trải qua một ngày kh...,"{""condition"": ""none"", ""severity_score"": 0}",<|user|>\nNhật ký: Hôm nay tôi đang trải qua m...,none,Nhật ký: Hôm nay tôi đang trải qua một ngày kh...,Ngôn ngữ và giọng điệu trong bài viết này của ...,0.0
2,"""Nếu như không có gì là thật? Nếu như tất cả c...","{""condition"": ""stress"", ""severity_score"": 3}","<|user|>\n""Nếu như không có gì là thật? Nếu nh...",stress,"""Nếu như không có gì là thật? Nếu như tất cả c...",Người viết bài đang trải qua một cơn đau thắt ...,3.0
3,Bây giờ tôi biết mọi người có thể nghĩ rằng đi...,"{""condition"": ""stress"", ""severity_score"": 4}",<|user|>\nBây giờ tôi biết mọi người có thể ng...,stress,Bây giờ tôi biết mọi người có thể nghĩ rằng đi...,Người viết bài đề cập đến việc sử dụng suboxon...,4.0
4,"Cập nhật - Cảm ơn mọi người, bây giờ tôi đã bì...","{""condition"": ""stress"", ""severity_score"": 3}","<|user|>\nCập nhật - Cảm ơn mọi người, bây giờ...",stress,"Cập nhật - Cảm ơn mọi người, bây giờ tôi đã bì...",Người viết bày tỏ cảm giác bị phản bội và buồn...,3.0
...,...,...,...,...,...,...,...
179,Hóa ra họ đã tạo cho ông bà tôi một hình ảnh r...,"{""condition"": ""none"", ""severity_score"": 0}",<|user|>\nHóa ra họ đã tạo cho ông bà tôi một ...,none,Hóa ra họ đã tạo cho ông bà tôi một hình ảnh r...,Người viết thảo luận về một tình huống khó khă...,0.0
180,Điều tích cực duy nhất mình nhận ra gần đây kh...,"{""condition"": ""depression"", ""severity_score"": 3}",<|user|>\nĐiều tích cực duy nhất mình nhận ra ...,depression,Điều tích cực duy nhất mình nhận ra gần đây kh...,Bài đăng đề cập đến những khía cạnh tích cực c...,3.0
181,Họ làm việc cùng nhau và tôi biết từ một người...,"{""condition"": ""stress"", ""severity_score"": 3}",<|user|>\nHọ làm việc cùng nhau và tôi biết từ...,stress,Họ làm việc cùng nhau và tôi biết từ một người...,Người viết bày tỏ khó khăn trong việc xử lý tì...,3.0
182,"Tại nơi trú ẩn, chúng tôi không được phép lưu ...","{""condition"": ""none"", ""severity_score"": 0}","<|user|>\nTại nơi trú ẩn, chúng tôi không được...",none,"Tại nơi trú ẩn, chúng tôi không được phép lưu ...",Bảng quảng cáo đang yêu cầu ý kiến về những lự...,0.0


In [ ]:
from tqdm import tqdm
import pandas as pd
import time

results = []

for _, row in tqdm(merged_test_df.iterrows(), total=len(merged_test_df)):
    user_input = row["post_vi"]
    expected_condition = row["condition"]
    expected_score = int(row["severity_score"])

    prompt = f"<|user|>\n{user_input}\n<|assistant|>\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Start timing
    start_time = time.perf_counter()

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)

    # End timing
    end_time = time.perf_counter()
    inference_time = end_time - start_time  # in seconds

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    parsed = extract_json_from_response(decoded)

    result = {
        "input": user_input,
        "expected_condition": expected_condition,
        "expected_score": expected_score,
        "predicted_condition": parsed["condition"],
        "predicted_score": parsed["severity_score"],
        "raw_output": decoded,
        "inference_time_sec": round(inference_time, 3),
    }

    results.append(result)

# Convert to DataFrame
results_df = pd.DataFrame(results)


100%|██████████| 184/184 [14:46<00:00,  4.82s/it]


In [ ]:
 results_df

,input,expected_condition,expected_score,predicted_condition,predicted_score,raw_output,inference_time_sec
0,"Tôi là một người trưởng thành mắc bệnh tâm lý,...",stress,4,stress,5,<|user|>\nTôi là một người trưởng thành mắc bệ...,5.343
1,Nhật ký: Hôm nay tôi đang trải qua một ngày kh...,none,0,none,0,<|user|>\nNhật ký: Hôm nay tôi đang trải qua m...,4.556
2,"""Nếu như không có gì là thật? Nếu như tất cả c...",stress,3,stress,4,"<|user|>\n""Nếu như không có gì là thật? Nếu nh...",4.963
3,Bây giờ tôi biết mọi người có thể nghĩ rằng đi...,stress,4,stress,4,<|user|>\nBây giờ tôi biết mọi người có thể ng...,4.779
4,"Cập nhật - Cảm ơn mọi người, bây giờ tôi đã bì...",stress,3,stress,4,"<|user|>\nCập nhật - Cảm ơn mọi người, bây giờ...",4.587
...,...,...,...,...,...,...,...
179,Hóa ra họ đã tạo cho ông bà tôi một hình ảnh r...,none,0,none,0,<|user|>\nHóa ra họ đã tạo cho ông bà tôi một ...,4.539
180,Điều tích cực duy nhất mình nhận ra gần đây kh...,depression,3,depression,3,<|user|>\nĐiều tích cực duy nhất mình nhận ra ...,5.446
181,Họ làm việc cùng nhau và tôi biết từ một người...,stress,3,stress,4,<|user|>\nHọ làm việc cùng nhau và tôi biết từ...,4.516
182,"Tại nơi trú ẩn, chúng tôi không được phép lưu ...",none,0,none,0,"<|user|>\nTại nơi trú ẩn, chúng tôi không được...",4.731


In [ ]:
condition_accuracy = accuracy_score(results_df["expected_condition"], results_df["predicted_condition"])

print("✅ Condition Classification Accuracy:", condition_accuracy)
print("\n📋 Classification Report (Condition):")
print(classification_report(results_df["expected_condition"], results_df["predicted_condition"]))
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(results_df["expected_score"], results_df["predicted_score"])
print("📉 Severity MAE:", round(mae, 2))


✅ Condition Classification Accuracy: 0.7989130434782609

📋 Classification Report (Condition):
              precision    recall  f1-score   support

  depression       0.82      0.92      0.87        36
        none       0.78      0.77      0.78        74
      stress       0.80      0.77      0.79        74

    accuracy                           0.80       184
   macro avg       0.80      0.82      0.81       184
weighted avg       0.80      0.80      0.80       184

📉 Severity MAE: 0.9
